## Train and test a classifier for each of the feature sets  for each dataset 

In [8]:
%reload_ext autoreload
%autoreload 2
import os
import pandas as pd
import pickle
import random
import numpy as np
import csv
import timeit
from collections import defaultdict, Counter
from pathlib import Path
from Bio import SeqIO
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split 
from sklearn.metrics import roc_auc_score
import sys
lib_dir = '../mylibs'
if lib_dir not in sys.path:
    sys.path.append(lib_dir)
%matplotlib inline    
    
import vhdb as vhdb
from featureset import FeatureSet
from dataset import DataSet

In [2]:
def get_confusion( y_test,y_pred):
    FN=0
    FP=0
    TP=0
    TN=0
    #missed = 0
    for y,yp in zip(y_test,y_pred):
        #print (y_true,yp,yp2,yprob)
        if y == True and yp == False:
            FN += 1
        elif y == False and yp == True:
            FP +=1
        elif y == True and yp == True:
            TP +=1
        else:
            TN +=1
    spec = round(TN/(TN+FP),2) if (TN + TP) else 'NA'
    sens = round(TP/(TP+FN),2) if (TP+FN) else 'NA'
    fdr  = round(FP/(TP+FP),2) if (TP+FP) else 'NA'
    
    #NB 8-6-2020 Need to change 'prec' in the below to 'Prec' otherwise causes error in results2CSV
    res  = { 'Acc':round((TP+TN)/(len(y_test)),2),'Spec':spec, 'Sens':sens,\
           'Prec':fdr,'TP':TP, 'TN':TN, 'FP':FP, 'FN':FN}
    return res


def test_prediction(fs,y_trn,y_tst):
    clf = make_pipeline(StandardScaler(),SVC(kernel='linear',probability=True) )
    clf.fit(fs.X_trn, y_trn)
    y_pred = clf.predict(fs.X_tst)
    y_pred_probs= clf.predict_proba(fs.X_tst)[:,1]
    AUC =round( roc_auc_score(y_tst, y_pred_probs),3)  
    confusion = get_confusion(y_tst,y_pred)
    confusion.update({'AUC':AUC})
    return confusion
       
        

### The input dataset files in ../inputs 
1. Bacteria_DNA.csv 
2. Euk_subsets.csv
3. Euk_all.csv
4. Euk_RNA.csv

### Load the input files to train and test
1. input/ datasets  
2. VHDB 

In [3]:
# Input file  
subsetfile = '../inputs/Euk_all.csv'
label_info = pd.read_csv(subsetfile)
subsets = label_info.apply(tuple, axis=1).tolist()
print(f'{len(subsets)} datasets : {subsets[0]}')
for s in subsets:
    print(s)
vhdbfile = '../inputs/VHDB_25_1_2019.p'
with open(vhdbfile, 'rb') as f:
    V_H = pickle.load( f)
hosts = V_H.hosts
viruses = V_H.viruses
print (f'{len(viruses)} viruses and {len(hosts)} hosts')


# Output file for the results
results_file = f'../results/{Path(subsetfile).stem}_results.csv '
print (f'Results will be saved in: {results_file}')



69 datasets : ('Bos taurus', 'species', 'Bovidae', 'family', 'all')
('Bos taurus', 'species', 'Bovidae', 'family', 'all')
('Bos', 'genus', 'Bovidae', 'family', 'all')
('Ovis aries', 'species', 'Bovidae', 'family', 'all')
('Sus scrofa', 'species', 'Mammalia', 'class', 'all')
('Suidae', 'family', 'Mammalia', 'class', 'all')
('Vespertilionidae', 'family', 'Chiroptera', 'order', 'all')
('Phyllostomidae', 'family', 'Chiroptera', 'order', 'all')
('Rhinolophus', 'genus', 'Chiroptera', 'order', 'all')
('Pteropus', 'genus', 'Pteropodidae', 'family', 'all')
('Pteropodidae', 'family', 'Chiroptera', 'order', 'all')
('Chiroptera', 'order', 'Mammalia', 'class', 'all')
('Equus caballus', 'species', 'Mammalia', 'class', 'all')
('Equus', 'genus', 'Mammalia', 'class', 'all')
('Equidae', 'family', 'Mammalia', 'class', 'all')
('Perissodactyla', 'order', 'Mammalia', 'class', 'all')
('Canis lupus', 'species', 'Carnivora', 'order', 'all')
('Canidae', 'family', 'Carnivora', 'order', 'all')
('Felis catus', 'sp

In [4]:

features = ['DNA','AA','PC','Domains']
kmer_lists = [[1,2,3,4,5,6,7,8,9], # dna 
              [1,2,3,4],           # aa
              [1,2,3,4,5,6] ,      #pc
              [0]]  
feature_sets = [f'{f}_{k}' for i,f in enumerate(features) for k in kmer_lists[i] ]


In [5]:
new_subsets = [subsets[24]]

In [17]:
feature_sets

['DNA_1',
 'DNA_2',
 'DNA_3',
 'DNA_4',
 'DNA_5',
 'DNA_6',
 'DNA_7',
 'DNA_8',
 'DNA_9',
 'AA_1',
 'AA_2',
 'AA_3',
 'AA_4',
 'PC_1',
 'PC_2',
 'PC_3',
 'PC_4',
 'PC_5',
 'PC_6',
 'Domains_0']

In [9]:
all_results =[]
for subset in new_subsets:
    print  (subset)
    data = DataSet(subset,V_H,feature_sets=feature_sets)
    (label,label_tax,pool,pool_tax,baltimore) = subset
    print  (label,label_tax,pool,pool_tax,baltimore)
    print((data.ds.groupby(['y','trn/tst']).count(),'\n'))
    mask = data.ds['trn/tst']=='train'
    y_train = np.asarray(data.ds[mask]['y'],dtype=int)
    y_test = np.asarray(data.ds[~mask
                               ]['y'],dtype=int)
    for fs in data.fs:
        results = test_prediction(fs,y_train,y_test)
        results.update ({'N': len(data.ds), 'features':fs.feature, 'k':fs.k})
        print(results)
        data.results2CSV (results,subset, results_file)
        all_results.append(results)
    results_df = pd.DataFrame(all_results)
   

('Homo sapiens', 'species', 'Hominidae', 'family', 'all')


C:\Users\NBOLLIG\Desktop\virus_host_predict\code\dataset.py:287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  train['trn/tst'] = train.apply(lambda row: 'train', axis = 1)
C:\Users\NBOLLIG\Desktop\virus_host_predict\code\dataset.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test['trn/tst'] = test.apply(lambda row: 'test', axis = 1)


train test split 57 20
     virus     refseqs  y trn/tst
0  1261247  [KC175394]  1   train
1   167321  [FJ445131]  1    test
2   318570  [AY843305]  1   train
3  1241967  [JX459901]  1    test
4   147684  [EF173415]  1   train
Adding feature sets  ['DNA_1', 'DNA_2', 'DNA_3', 'DNA_4', 'DNA_5', 'DNA_6', 'DNA_7', 'DNA_8', 'DNA_9', 'AA_1', 'AA_2', 'AA_3', 'AA_4', 'PC_1', 'PC_2', 'PC_3', 'PC_4', 'PC_5', 'PC_6', 'Domains_0']
adding fs DNA_1
FS get sequences for length ds 77
Missing: 
[]
> c:\users\nbollig\desktop\virus_host_predict\code\featureset.py(33)__init__()
     31         from IPython.core.debugger import set_trace
     32         set_trace()
---> 33         print('FS get feature names', f_s, len(seqs))
     34         self.feature_names = self.get_feature_names(seqs)
     35         print('FS get X','len fs',len(self.feature_names))

ipdb> seqs
defaultdict(<class 'str'>, {'1261247': 'gaatgaagatggcgtctaacgacgcttccgctgccgctgttgctaacagcaacaacgacaccgcaaaatcttcaagtgacaaaatgttttctaacatggc

ipdb> seqs.keys()
dict_keys(['1261247', '167321', '318570', '1241967', '147684', '126794', '546980', '222887', '11039', '68558', '1195616', '2170195', '321147', '1306931', '1261232', '64307', '185935', '484894', '1261210', '1803956', '61673', '1261195', '546987', '103914', '10325', '11292', '994672', '57482', '1315259', '1650736', '185923', '333763', '1535290', '2065055', '167327', '767521', '626181', '11021', '11168', '1891737', '1123958', '93986', '1330996', '687372', '332937', '1743411', '1604875', '2170204', '1604874', '1891735', '11723', '1236400', '221703', '33748', '742920', '175567', '198503', '687362', '742921', '928213', '687341', '1891736', '188763', '864686', '687353', '1454033', '2035845', '928211', '743290', '1891739', '927576', '928214', '113194', '1891738', '687343', '82664', '1762023'])
ipdb> c
FS get feature names DNA_1 77
FS get X len fs 4
77 77 4 (77, 4)
adding fs DNA_2
FS get sequences for length ds 77
Missing: 
[]
> c:\users\nbollig\desktop\virus_host_predict\code

BdbQuit: 

In [10]:
vhdb.__file__

'C:\\Users\\NBOLLIG\\Desktop\\virus_host_predict\\code\\vhdb.py'

In [ ]:
n